In [7]:
%%bash
conda install requests

Channels:
 - defaults
 - conda-forge
Platform: linux-64
Solving environment: ...working... done

## Package Plan ##

  environment location: /home/polibenitez/anaconda3/envs/complxt

  added / updated specs:
    - requests


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    brotli-python-1.0.9        |  py313h6a678d5_9         356 KB
    certifi-2024.12.14         |  py313h06a4308_0         159 KB
    idna-3.7                   |  py313h06a4308_0         104 KB
    pysocks-1.7.1              |  py313h06a4308_0          35 KB
    requests-2.32.3            |  py313h06a4308_1         125 KB
    urllib3-2.3.0              |  py313h06a4308_0         244 KB
    ------------------------------------------------------------
                                           Total:        1023 KB

The following NEW packages will be INSTALLED:

  brotli-python      pkgs/main/linux-64::brotli-python-1.0.9-py

In [ ]:
import requests
import json

def llamar_deepseek(prompt, modelo="deepseek-r1:14b", url_base="http://localhost:11434/api/generate"):
    """
    Función para hacer llamadas a un modelo de DeepSeek levantado localmente usando Ollama.

    :param prompt: El texto de entrada que se enviará al modelo.
    :param modelo: El nombre del modelo que deseas usar (por defecto es "deepseek").
    :param url_base: La URL base del servidor local de Ollama (por defecto es "http://localhost:11434/api/generate").
    :return: La respuesta generada por el modelo.
    """
    # Configura los datos que se enviarán en la solicitud POST
    datos = {
        "model": modelo,
        "prompt": prompt,
        "stream": False  # Si deseas recibir la respuesta en streaming, cambia a True
    }

    # Realiza la solicitud POST al servidor local de Ollama
    respuesta = requests.post(url_base, json=datos)

    # Verifica si la solicitud fue exitosa
    if respuesta.status_code == 200:
        # Decodifica la respuesta JSON
        respuesta_json = respuesta.json()
        return respuesta_json.get("response", "No se recibió respuesta del modelo.")
    else:
        # Si la solicitud no fue exitosa, lanza una excepción con el código de estado
        raise Exception(f"Error en la solicitud: {respuesta.status_code} - {respuesta.text}")

# Ejemplo de uso
if __name__ == "__main__":
    prompt =  """
                        Act as an address parsing and standardization service. Given a list of addresses, extract and standardize the following components:
                        - street: Full street address including number
                        - city: City/town/municipality name
                        - postcode: Postal/ZIP code
                        - country: Full country name
                        - nuts3: NUTS3 region code if available
                        
                        Requirements:
                        - Use DOUBLE QUOTES for JSON properties
                        - Standardize addresses while preserving original information
                        - Fill in missing components when possible based on context
                        - Return null for truly missing/unidentifiable components
                        
                        Example output format:
                        [{
                            "street": "123 Main Street",
                            "city": "Brussels",
                            "postcode": "1000",
                            "country": "Belgium",
                            "nuts3": "BE100"
                        }]
                    """
    try:
        respuesta = llamar_deepseek(prompt)
        print("Respuesta del modelo:", respuesta)
    except Exception as e:
        print(e)

In [3]:
import time

# Función para convertir segundos a horas, minutos y segundos
def format_time(seconds):
    hours = int(seconds // 3600)  # 1 hora = 3600 segundos
    seconds %= 3600
    minutes = int(seconds // 60)  # 1 minuto = 60 segundos
    seconds %= 60
    return f"{hours}h {minutes}m {seconds:.2f}s"


print(f'Process finished for file. Time elapsed: {format_time(6080)}')

Process finished for file. Time elapsed: 1h 41m 20.00s


In [2]:
import pandas as pd

df = pd.read_parquet('patents_complx_gb/gb.parquet')
print(len(df))
df1 = df.head(1000)
df1.to_parquet('patents_complx_gb/gb1.parquet')

832589


In [1]:
import pyarrow.parquet as pq
import pyarrow as pa

# Leer el archivo Parquet
table = pq.read_table('patents_complx_gb/gb.parquet')

# Calcular el tamaño de cada parte
n = len(table)
part_size = n // 3

# Dividir la tabla en 3 partes
part1 = table.slice(0, part_size)
part2 = table.slice(part_size, part_size)
part3 = table.slice(2 * part_size)

# Guardar cada parte
pq.write_table(part1, 'patents_complx_gb/parte1.parquet')
pq.write_table(part2, 'patents_complx_gb/parte2.parquet')
pq.write_table(part3, 'patents_complx_gb/parte3.parquet')